In [1]:
import sys
sys.path.append('../')

In [2]:
import numpy as np
import lxmls.readers.sentiment_reader as srs 
scr = srs.SentimentCorpus("books")
train_x = scr.train_X.T
train_y = scr.train_y[:, 0]
test_x = scr.test_X.T
test_y = scr.test_y[:, 0]

2000
1600


In [3]:
train_x.shape

(13989, 1600)

In [4]:
# Neural network modules
import lxmls.deep_learning.mlp as dl 
import lxmls.deep_learning.sgd as sgd 
# Model parameters
geometry = [train_x.shape[0], 20, 2] 
print (geometry)
actvfunc = ['sigmoid', 'softmax']
# Instantiate model
mlp      = dl.NumpyMLP(geometry, actvfunc)

[13989, 20, 2]


In [5]:
# Model parameters
n_iter = 5
bsize  = 5
lrate  = 0.01
# Train
sgd.SGD_train(mlp, n_iter, bsize=bsize, lrate=lrate, train_set=(train_x, train_y))
acc_train = sgd.class_acc(mlp.forward(train_x), train_y)[0]
acc_test = sgd.class_acc(mlp.forward(test_x), test_y)[0]
print "MLP (%s) Amazon Sentiment Accuracy train: %f test: %f" % (geometry, acc_train,
acc_test)

Batch 320/320 (100%)   Epoch  1/ 5 in 2.41 seg
Batch 320/320 (100%)   Epoch  2/ 5 in 2.49 seg
Batch 320/320 (100%)   Epoch  3/ 5 in 2.28 seg
Batch 320/320 (100%)   Epoch  4/ 5 in 2.12 seg
Batch 320/320 (100%)   Epoch  5/ 5 in 2.38 seg
 
MLP ([13989, 20, 2]) Amazon Sentiment Accuracy train: 0.964375 test: 0.780000


## Introduction to Theano :(

In [6]:
# Numpy code
x = test_x
W1, b1 = mlp.params[:2]
z1 = np.dot(W1, x) + b1
tilde_z1 = 1/(1+np.exp(-z1))  # Non-linear transformation

In [7]:
# Theano code.
# NOTE: We use undescore to denote symbolic equivalents to Numpy variables. # This is no Python convention!.
import theano
import theano.tensor as T
_x = T.matrix('x')
_W1 = theano.shared(value=W1, name='W1', borrow=True)
_b1 = theano.shared(value=b1, name='b1', borrow=True, broadcastable=(False, True))
_z1            = T.dot(_W1, _x) + _b1
_tilde_z1      = T.nnet.sigmoid(_z1)
# Keep in mind that naming variables is useful when debugging
_z1.name       = 'z1'
_tilde_z1.name = 'tilde_z1'

In [8]:
# Perceptron computation graph
theano.printing.debugprint(_tilde_z1)

sigmoid [id A] 'tilde_z1'   
 |Elemwise{add,no_inplace} [id B] 'z1'   
   |dot [id C] ''   
   | |W1 [id D]
   | |x [id E]
   |b1 [id F]


In [9]:
layer1 = theano.function([_x], _tilde_z1)

In [10]:
# Check Numpy and Theano match
assert np.allclose(tilde_z1, layer1(x.astype(theano.config.floatX))), \
"Numpy and Theano Perceptrons are different"

In [11]:
train_x = train_x.astype(theano.config.floatX)
train_y = train_y.astype('int32')